<h1> Using Labeled Data to Train a Network, Use The Network to Label New Videos, and Create Trajectories </h1> 
<p> Use another notebook create a project, extract frames, and label training/testing data </p>

# DLC with docker
This notebook illustrates how to use the Docker container to:
- train a network
- evaluate a network
- analyze a novel video

many of the functions have additional parameters / complexity, see the DLC docs for more inf on each.

This assumes you already have a project folder with labeled data! 


## info about the Docker Environment:
Docker is essentially a better alternative to virtual machines. It is able to containerize applications in a way that keeps them seperate the OS and other software. 

For tensorflow / deeplab cut, docker is critical for two reasons. 
- It 'claims' the GPU, making it closed for use by other processes.
- It queries the GPU properly, in some cases tf (and the python kernal itself) does not know how to properly communincate with the GPU on windows systems.

In [2]:
# make sure you graphic driver is accessable
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:32_Central_Daylight_Time_2017
Cuda compilation tools, release 9.0, V9.0.176


In [3]:
# check the tensorflow version
import tensorflow as tf
tf.__version__

'1.14.0'

<h1> The following is very important as it allow the current GPU process to grow dynamically </h1>
Without this option tf will likely run out of VRAM when trying to update the weight tensor. In theory, these options could cause the GPU to run out of memory entirely, but there is no other way to allow training to complete successfully. 

In [4]:
#allow video memory growth as network expands to avoid convolutional network errors
TF_FORCE_GPU_ALLOW_GROWTH = True
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [5]:
#let's make sure we see a GPU:
#tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3929145459348125217, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5335897831080857810
 physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:41:00.0, compute capability: 7.5", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15612114841078075414
 physical_device_desc: "device: 1, name: GeForce RTX 2080, pci bus id: 0000:81:00.0, compute capability: 7.5"]

## Start here for training DeepLabCut and analyzing new videos!
<p><br>If the first imports fail, there is again - sadly - an issue with you enviroment. Make sure all packages beside DLC are installed via conda. </p> 

In [6]:
#GUIs don't work on in Docker (or the cloud), so label your data locally on your computer! 
#This notebook is for you to train and run video analysis!
import os

In [7]:
# now we are ready to train!
#should see version 2.0.8
import deeplabcut
deeplabcut.__version__

'2.0.8'

<h1> change to your path: </h1>
<p> this should be the same path as the one in the createDLCproject notebook. The path is the path to the config.yaml file, not the project directory itself </p> 

In [10]:
path_config_file = r'F:\MysoreData\nbk\TEST_50fps_mitg05_042120_cam1-spencer loggia-2020-04-24\config.yaml' #change to yours!

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

In [9]:
deeplabcut.create_training_dataset(path_config_file,Shuffles=[1])

F:\MysoreData\nbk\TEST_50fps_mitg05_042120_cam1-spencer loggia-2020-04-24\training-datasets\iteration-0\UnaugmentedDataSet_TEST_50fps_mitg05_042120_cam1Apr24  already exists!
F:\MysoreData\nbk\TEST_50fps_mitg05_042120_cam1-spencer loggia-2020-04-24\labeled-data\mitg05-839--04212020093253.avi_clip_10/CollectedData_spencer loggia.h5  not found (perhaps not annotated)
F:\MysoreData\nbk\TEST_50fps_mitg05_042120_cam1-spencer loggia-2020-04-24\dlc-models\iteration-0\TEST_50fps_mitg05_042120_cam1Apr24-trainset95shuffle1  already exists!
F:\MysoreData\nbk\TEST_50fps_mitg05_042120_cam1-spencer loggia-2020-04-24\dlc-models\iteration-0\TEST_50fps_mitg05_042120_cam1Apr24-trainset95shuffle1//train  already exists!
F:\MysoreData\nbk\TEST_50fps_mitg05_042120_cam1-spencer loggia-2020-04-24\dlc-models\iteration-0\TEST_50fps_mitg05_042120_cam1Apr24-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


### now go edit the pose_cfg.yaml to make display_iters: low (i.e. 10), and save_iters: 500

Now it is the time to start training the network!

## Start training
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#reset in case you started a session before...
#tf.reset_default_graph()

deeplabcut.train_network(path_config_file, shuffle=1, saveiters=1000, displayiters=10, gputouse=1)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.3M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....


Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['bodypart1', 'bodypart2', 'bodypart3'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_TEST_50fps_mitg05_042120_cam1Apr24\\TEST_50fps_mitg05_042120_cam1_spencer '
            'loggia95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\pbs-mysorelab\\.conda\\envs\\nbkGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\Una

Starting with standard pose-dataset loader.


W0424 15:39:40.575442 53732 deprecation.py:323] From C:\Users\pbs-mysorelab\.conda\envs\nbkGPU\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0424 15:39:40.622321 53732 deprecation.py:323] From C:\Users\pbs-mysorelab\.conda\envs\nbkGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\nnet\losses.py:38: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0424 15:39:56.121945 53732 deprecation.py:323] From C:\Users\pbs-mysorelab\.conda\envs\nbkGPU\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for 

Display_iters overwritten as 10
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'F:\\MysoreData\\nbk\\TEST_50fps_mitg05_042120_cam1-spencer loggia-2020-04-24\\dlc-models\\iteration-0\\TEST_50fps_mitg05_042120_cam1Apr24-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 40

iteration: 10 loss: 0.2508 lr: 0.005
iteration: 20 loss: 0.0472 lr: 0.005
iteration: 30 loss: 0.0348 lr: 0.005
iteration: 40 loss: 0.0346 lr: 0.005
iteration: 50 loss: 0.0282 lr: 0.005
iteration: 60 loss: 0.0262 lr: 0.005
iteration: 70 loss: 0.0265 lr: 0.005
iteration: 80 loss: 0.0265 lr: 0.005
iteration: 90 loss: 0.0297 lr: 0.005
iteration: 100 loss: 0.0267 lr: 0.005
iteration: 110 loss: 0.0242 lr: 0.005
iteration: 120 loss: 0.0240 lr: 0.005
iteration: 130 loss: 0.0248 lr: 0.005
iteration: 140 loss: 0.0190 lr: 0.005
iteration: 150 loss: 0.0277 lr: 0.005
iteration: 160 loss: 0.0229 lr: 0.005
iteration: 170 loss: 0.0196 lr: 0.005
iteration: 180 loss: 0.0229 lr: 0.005
iteration: 190 loss: 0.0253 lr: 0.005
iteration: 200 loss: 0.0192 lr: 0.005
iteration: 210 loss: 0.0261 lr: 0.005
iteration: 220 loss: 0.0223 lr: 0.005
iteration: 230 loss: 0.0209 lr: 0.005
iteration: 240 loss: 0.0186 lr: 0.005
iteration: 250 loss: 0.0205 lr: 0.005
iteration: 260 loss: 0.0256 lr: 0.005
iteration: 270 loss: 

...

iteration: 79650 loss: 0.0024 lr: 0.02
iteration: 79660 loss: 0.0016 lr: 0.02
iteration: 79670 loss: 0.0023 lr: 0.02
iteration: 79680 loss: 0.0013 lr: 0.02
iteration: 79690 loss: 0.0012 lr: 0.02
iteration: 79700 loss: 0.0020 lr: 0.02
iteration: 79710 loss: 0.0014 lr: 0.02
iteration: 79720 loss: 0.0017 lr: 0.02
iteration: 79730 loss: 0.0019 lr: 0.02
iteration: 79740 loss: 0.0011 lr: 0.02
iteration: 79750 loss: 0.0014 lr: 0.02
iteration: 79760 loss: 0.0011 lr: 0.02
iteration: 79770 loss: 0.0017 lr: 0.02
iteration: 79780 loss: 0.0016 lr: 0.02
iteration: 79790 loss: 0.0013 lr: 0.02
iteration: 79800 loss: 0.0015 lr: 0.02
iteration: 79810 loss: 0.0014 lr: 0.02
iteration: 79820 loss: 0.0019 lr: 0.02
iteration: 79830 loss: 0.0017 lr: 0.02
iteration: 79840 loss: 0.0013 lr: 0.02
iteration: 79850 loss: 0.0012 lr: 0.02
iteration: 79860 loss: 0.0019 lr: 0.02
iteration: 79870 loss: 0.0015 lr: 0.02
iteration: 79880 loss: 0.0017 lr: 0.02
iteration: 79890 loss: 0.0015 lr: 0.02
iteration: 79900 loss: 0.

iteration: 81750 loss: 0.0018 lr: 0.02
iteration: 81760 loss: 0.0012 lr: 0.02
iteration: 81770 loss: 0.0014 lr: 0.02
iteration: 81780 loss: 0.0015 lr: 0.02
iteration: 81790 loss: 0.0017 lr: 0.02
iteration: 81800 loss: 0.0021 lr: 0.02
iteration: 81810 loss: 0.0016 lr: 0.02
iteration: 81820 loss: 0.0016 lr: 0.02
iteration: 81830 loss: 0.0023 lr: 0.02
iteration: 81840 loss: 0.0014 lr: 0.02
iteration: 81850 loss: 0.0015 lr: 0.02
iteration: 81860 loss: 0.0012 lr: 0.02
iteration: 81870 loss: 0.0018 lr: 0.02
iteration: 81880 loss: 0.0016 lr: 0.02
iteration: 81890 loss: 0.0016 lr: 0.02
iteration: 81900 loss: 0.0020 lr: 0.02
iteration: 81910 loss: 0.0014 lr: 0.02
iteration: 81920 loss: 0.0013 lr: 0.02
iteration: 81930 loss: 0.0015 lr: 0.02
iteration: 81940 loss: 0.0011 lr: 0.02
iteration: 81950 loss: 0.0017 lr: 0.02
iteration: 81960 loss: 0.0015 lr: 0.02
iteration: 81970 loss: 0.0019 lr: 0.02
iteration: 81980 loss: 0.0012 lr: 0.02
iteration: 81990 loss: 0.0023 lr: 0.02
iteration: 82000 loss: 0.

iteration: 83850 loss: 0.0024 lr: 0.02
iteration: 83860 loss: 0.0020 lr: 0.02
iteration: 83870 loss: 0.0015 lr: 0.02
iteration: 83880 loss: 0.0013 lr: 0.02
iteration: 83890 loss: 0.0022 lr: 0.02
iteration: 83900 loss: 0.0014 lr: 0.02
iteration: 83910 loss: 0.0022 lr: 0.02
iteration: 83920 loss: 0.0015 lr: 0.02
iteration: 83930 loss: 0.0022 lr: 0.02
iteration: 83940 loss: 0.0017 lr: 0.02
iteration: 83950 loss: 0.0011 lr: 0.02
iteration: 83960 loss: 0.0016 lr: 0.02
iteration: 83970 loss: 0.0020 lr: 0.02
iteration: 83980 loss: 0.0014 lr: 0.02
iteration: 83990 loss: 0.0013 lr: 0.02
iteration: 84000 loss: 0.0012 lr: 0.02
iteration: 84010 loss: 0.0019 lr: 0.02
iteration: 84020 loss: 0.0016 lr: 0.02
iteration: 84030 loss: 0.0016 lr: 0.02
iteration: 84040 loss: 0.0019 lr: 0.02
iteration: 84050 loss: 0.0019 lr: 0.02
iteration: 84060 loss: 0.0022 lr: 0.02
iteration: 84070 loss: 0.0017 lr: 0.02
iteration: 84080 loss: 0.0017 lr: 0.02
iteration: 84090 loss: 0.0013 lr: 0.02
iteration: 84100 loss: 0.

iteration: 85950 loss: 0.0016 lr: 0.02
iteration: 85960 loss: 0.0016 lr: 0.02
iteration: 85970 loss: 0.0020 lr: 0.02
iteration: 85980 loss: 0.0011 lr: 0.02
iteration: 85990 loss: 0.0014 lr: 0.02
iteration: 86000 loss: 0.0015 lr: 0.02
iteration: 86010 loss: 0.0015 lr: 0.02
iteration: 86020 loss: 0.0010 lr: 0.02
iteration: 86030 loss: 0.0013 lr: 0.02
iteration: 86040 loss: 0.0022 lr: 0.02
iteration: 86050 loss: 0.0013 lr: 0.02
iteration: 86060 loss: 0.0021 lr: 0.02
iteration: 86070 loss: 0.0010 lr: 0.02
iteration: 86080 loss: 0.0011 lr: 0.02
iteration: 86090 loss: 0.0015 lr: 0.02
iteration: 86100 loss: 0.0016 lr: 0.02
iteration: 86110 loss: 0.0016 lr: 0.02
iteration: 86120 loss: 0.0018 lr: 0.02
iteration: 86130 loss: 0.0013 lr: 0.02
iteration: 86140 loss: 0.0016 lr: 0.02
iteration: 86150 loss: 0.0013 lr: 0.02
iteration: 86160 loss: 0.0016 lr: 0.02
iteration: 86170 loss: 0.0021 lr: 0.02
iteration: 86180 loss: 0.0014 lr: 0.02
iteration: 86190 loss: 0.0015 lr: 0.02
iteration: 86200 loss: 0.

iteration: 88050 loss: 0.0013 lr: 0.02
iteration: 88060 loss: 0.0013 lr: 0.02
iteration: 88070 loss: 0.0013 lr: 0.02
iteration: 88080 loss: 0.0017 lr: 0.02
iteration: 88090 loss: 0.0021 lr: 0.02
iteration: 88100 loss: 0.0020 lr: 0.02
iteration: 88110 loss: 0.0018 lr: 0.02
iteration: 88120 loss: 0.0012 lr: 0.02
iteration: 88130 loss: 0.0024 lr: 0.02
iteration: 88140 loss: 0.0015 lr: 0.02
iteration: 88150 loss: 0.0014 lr: 0.02
iteration: 88160 loss: 0.0017 lr: 0.02
iteration: 88170 loss: 0.0014 lr: 0.02
iteration: 88180 loss: 0.0015 lr: 0.02
iteration: 88190 loss: 0.0018 lr: 0.02
iteration: 88200 loss: 0.0015 lr: 0.02
iteration: 88210 loss: 0.0018 lr: 0.02
iteration: 88220 loss: 0.0011 lr: 0.02
iteration: 88230 loss: 0.0013 lr: 0.02
iteration: 88240 loss: 0.0014 lr: 0.02
iteration: 88250 loss: 0.0015 lr: 0.02
iteration: 88260 loss: 0.0014 lr: 0.02
iteration: 88270 loss: 0.0017 lr: 0.02
iteration: 88280 loss: 0.0017 lr: 0.02
iteration: 88290 loss: 0.0015 lr: 0.02
iteration: 88300 loss: 0.

### A couple tips for possible troubleshooting (1): 

if you get **permission errors** when you run this step (above), first check if the weights downloaded. As some docker containers might not have privileges for this (it can be user specific). They should be under 'init_weights' (see path in the pose_cfg.yaml file). You can enter the DOCKER in the terminal:

see more here: https://github.com/MMathisLab/Docker4DeepLabCut2.0#using-the-docker-for-training-and-video-analysis

### Troubleshooting (2): 
if it appears the training does not start (i.e. "Starting training..." does not print immediately),
then you have another session running on your GPU. Go check "nvidia-smi" and look at the process names. You can only have 1 per GPU!)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(path_config_file)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

## There is an optional refinement step
- if your pixel errors are not low enough, use DLC docs on how to refine yur network!
- You will need to adjust the labels outside of DOCKER! (you can use the createDLCproject notebook) 
-  see DLC protocol instructions on how to refine your data!

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
videofile_path = [r'F:\MysoreData\nbk\cam7\raw_video\c_13817_409.avi'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
#for making interactive plots.
#deeplabcut.plot_trajectories(path_config_file,videofile_path, plotting=True)

deeplabcut.plot_trajectories(path_config_file,videofile_path,showfigures=True)